The goal here is to check the outputs of `bash_match_all_pdfs.sh` which runs `match_pdfs.py`.

In [1]:
import matplotlib
import numpy as np
import pandas as pd
import os
from astropy.table import Table
import time
from astropy.io import fits

#### Read in the saved, matched data for different fields for different photo-z algorithms.

In [2]:
pdfs_path = '/global/cscratch1/sd/awan/hsc_matched_pdfs/'

In [3]:
%%time
for field in  ['wide_aegis', 'wide_gama09h', 'wide_gama15h', 'wide_hectomap', 'wide_vvds', \
               'wide_wide12h', 'wide_xmmlss', 'deep_cosmos', 'deep_elaisn1', 'deep_xmmlss', 'deep_deep23']:
    print('\n ################################################################# ')
    print('\nWorking with %s'%field)
    print('\n ################################################################# ')
    for alg in ['ephor', 'ephor_ab', 'demp', 'frankenz']:
        print('\nWorking with %s'%alg)
        print('\n ----------------------------------------------------------------- ')
        filename= 'matched_pdfs_ids_bins_%s_%s.fits'%(field, alg)
        print('Reading in %s\n'%filename)
        hdul = fits.open('%s/%s'%(pdfs_path, filename))
        
        print(hdul.info())
        print('\n## Primary header:\nField: %s\nMatched with %s\nNpatches: %s'%(hdul[0].header['FIELD'],
                                                                                hdul[0].header['MATCHCAT'],
                                                                                hdul[0].header['NPATCH']))
        print('\n## Have data with shape %s, %s objects and %s bins.\n'%(np.shape(hdul[1].data['pdf']),
                                                                           len(hdul[1].data['object_id']),
                                                                           len(hdul[2].data['bins'])))


 ################################################################# 

Working with wide_aegis

 ################################################################# 

Working with ephor

 ----------------------------------------------------------------- 
Reading in matched_pdfs_ids_bins_wide_aegis_ephor.fits

Filename: /global/cscratch1/sd/awan/hsc_matched_pdfs//matched_pdfs_ids_bins_wide_aegis_ephor.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       7   ()      
  1                1 BinTableHDU     12   145185R x 2C   [K, 141E]   
  2                1 BinTableHDU     10   141R x 1C   [E]   
None

## Primary header:
Field: wide_aegis
Matched with WIDE_AEGIS_Catalog_i24.50.fits
Npatches: 4

## Have data with shape (145185, 141), 145185 objects and 141 bins.


Working with ephor_ab

 ----------------------------------------------------------------- 
Reading in matched_pdfs_ids_bins_wide_aegis_ephor_ab.fits

Filename: /global/cscratch1/sd

#### Check whether the object ids saved in the newly-created fits files have the same object_ids as in the processed catalogs.

In [4]:
datapath = '/global/cscratch1/sd/damonge/HSC/HSC_processed'

In [5]:
%%time
for field in  ['wide_aegis', 'wide_gama09h', 'wide_gama15h', 'wide_hectomap', 'wide_vvds', \
               'wide_wide12h', 'wide_xmmlss', 'deep_cosmos', 'deep_elaisn1', 'deep_xmmlss', 'deep_deep23']:
    print('\n ################################################################# ')
    print('\nWorking with %s'%field)
    print('\n ################################################################# ')
    # read in the cleaned catalog to get the object IDs
    for j, folder in enumerate([f for f in os.listdir(datapath) \
                                if not f.__contains__('.fits') and \
                                f.__contains__(field.upper())]):
        # read only the Catalog fits file for the field
        for i, cat_file in enumerate([f for f in os.listdir('%s/%s'%(datapath, folder)) \
                                      if f.__contains__('Catalog')]):
            if i>0:
                raise ValueError('Something is wrong. Have more than one Catalog file in %s/%s'%(datapath,
                                                                                                 folder))

            print('\nReading in %s'%cat_file)
            dat = Table.read('%s/%s/%s'%(datapath, folder, cat_file), format='fits')
        if j==0:
            hscdata = dat.to_pandas()
            print('Data read in. Shape: %s'%(np.shape(hscdata),))
        else:
            raise ValueError('Something is not right. Have more than one folder for %s: %s'%(field,
                                                                                             folder))
            
        print('')
    
    for alg in ['ephor', 'ephor_ab', 'demp', 'frankenz']:
        print('\n ----------------------------------------------------------------- ')
        print('\nReading in matched data for %s'%alg)
        
        filename= 'matched_pdfs_ids_bins_%s_%s.fits'%(field, alg)
        print('Reading in %s'%filename)
        hdul = fits.open('%s/%s'%(pdfs_path, filename))
        
        print('IDs match: %s\n'%(np.unique(hscdata['object_id']==hdul[1].data['object_id'])))


 ################################################################# 

Working with wide_aegis

 ################################################################# 

Reading in WIDE_AEGIS_Catalog_i24.50.fits
Data read in. Shape: (145185, 110)


 ----------------------------------------------------------------- 

Reading in matched data for ephor
Reading in matched_pdfs_ids_bins_wide_aegis_ephor.fits
IDs match: [ True]


 ----------------------------------------------------------------- 

Reading in matched data for ephor_ab
Reading in matched_pdfs_ids_bins_wide_aegis_ephor_ab.fits
IDs match: [ True]


 ----------------------------------------------------------------- 

Reading in matched data for demp
Reading in matched_pdfs_ids_bins_wide_aegis_demp.fits
IDs match: [ True]


 ----------------------------------------------------------------- 

Reading in matched data for frankenz
Reading in matched_pdfs_ids_bins_wide_aegis_frankenz.fits
IDs match: [ True]


 ##############################

Okay, things appear to be working.